In [1]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import time

import cntk as C

%matplotlib inline

In [2]:
if 'TEST_DEVICE' in os.environ:
    if os.environ['TEST_DEVICE'] == 'cpu':
        C.device.try_set_default_device(C.device.cpu())
        print("Using CPU")
    else:
        C.device.try_set_default_device(C.device.gpu(0))
        print("Using GPU")

In [3]:
C.device.try_set_default_device(C.device.gpu(0))

True

## Data reading

In [4]:
# Ensure we always get the same amount of randomness
np.random.seed(0)
C.cntk_py.set_fixed_random_seed(1)
C.cntk_py.force_deterministic_algorithms()

# Define the data dimensions
input_dim_model = (1, 64, 64)    # images are 64 x 64 with 1 channel of color (gray)
input_dim = 64*64                # used by readers to treat input data as a vector
num_output_classes = 4

In [5]:
# Read a CTF formatted text (as mentioned above) using the CTF deserializer from a file
def create_reader(path, is_training, input_dim, num_label_classes):
    
    ctf = C.io.CTFDeserializer(path, C.io.StreamDefs(
          labels=C.io.StreamDef(field='labels', shape=num_label_classes, is_sparse=False),
          features=C.io.StreamDef(field='features', shape=input_dim, is_sparse=False)))
                          
    return C.io.MinibatchSource(ctf,
        randomize = is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)

In [102]:
data_dir = "data"
train_file=os.path.join(data_dir, "Train-64x64x15000_cntk_text.txt")
test_file=os.path.join(data_dir, "Test-64x64x5000_cntk_text.txt")
#train_file=os.path.join(data_dir, "Train-64x64x80000_20180718_095800.txt")
#test_file=os.path.join(data_dir, "Test-64x64x20000_20180718_095800.txt")


## CNN Model Creation

In [7]:
x = C.input_variable(input_dim_model)
y = C.input_variable(num_output_classes)
print('x: ', x)
print('y:', y)

x:  Input('Input3', [#], [1 x 64 x 64])
y: Input('Input4', [#], [4])


# function to build model

def create_model_lr(features):
    with C.layers.default_options(init = C.glorot_uniform()):
        r = C.layers.Dense(num_output_classes, activation = None)(features)
        return r

num_hidden_layers = 2
hidden_layers_dim = 400
def create_model_mlp(features):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.ops.relu):
            h = features
            for _ in range(num_hidden_layers):
                h = C.layers.Dense(hidden_layers_dim)(h)
            r = C.layers.Dense(num_output_classes, activation = None)(h)
            #r = C.layers.Dense(num_output_classes, activation = C.ops.sigmoid)(h)
            return r    
    
def create_model_cnn1(features):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
            h = features
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=8, 
                                       strides=(2,2), 
                                       pad=True, name='first_conv')(h)
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=16, 
                                       strides=(2,2), 
                                       pad=True, name='second_conv')(h)
            r = C.layers.Dense(num_output_classes, activation=None, name='classify')(h)
            return r
        
def create_model_v1(features):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
            h = features
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=8, 
                                       strides=(2,2), 
                                       pad=True, name='conv1')(h)
#            h = C.layers.AveragePooling((5,5), strides=(2,2))(h)
#            h = C.layers.Convolution2D(filter_shape=(5,5), 
#                                       num_filters=16, 
#                                       strides=(1,1),
#                                       pad=True, name='conv2')(h)
#            h = C.layers.AveragePooling((5,5), strides=(2,2))(h)
            r = C.layers.Dense(num_output_classes, activation=None, name='classify')(h)
            return r
    
def create_model_v1_1(features):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.relu):
            h = features

            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=8,
                                       strides=(1,1),
                                       pad=True, name="conv1")(h)
            h = C.layers.MaxPooling(filter_shape=(2,2),
                                    strides=(2,2), name="max1")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=16,
                                       strides=(1,1),
                                       pad=True, name="conv2")(h)
            h = C.layers.MaxPooling(filter_shape=(3,3),
                                    strides=(3,3), name="max2")(h)
            r = C.layers.Dense(num_output_classes, activation = None, name="classify")(h)
            return r


def create_model_v1_2(features):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.relu):
            h = features

            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=8,
                                       strides=(1,1),
                                       pad=True, name="conv1")(h)
            h = C.layers.MaxPooling(filter_shape=(2,2),
                                    strides=(2,2), name="max1")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=16,
                                       strides=(1,1),
                                       pad=True, name="conv2")(h)
            h = C.layers.MaxPooling(filter_shape=(3,3),
                                    strides=(3,3), name="max2")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=32,
                                       strides=(1,1),
                                       pad=True, name="conv3")(h)
            h = C.layers.MaxPooling(filter_shape=(4,4),
                                    strides=(4,4), name="max3")(h)
            r = C.layers.Dense(num_output_classes, activation = None, name="classify")(h)
            return r


def create_model_v1_3(features):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.relu):
            h = features

            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=8,
                                       strides=(1,1),
                                       pad=True, name="conv1")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=16,
                                       strides=(1,1),
                                       pad=True, name="conv2")(h)
            h = C.layers.MaxPooling(filter_shape=(2,2),
                                    strides=(2,2), name="max1")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=32,
                                       strides=(1,1),
                                       pad=True, name="conv3")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=64,
                                       strides=(1,1),
                                       pad=True, name="conv4")(h)
            h = C.layers.MaxPooling(filter_shape=(3,3),
                                    strides=(3,3), name="max2")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=128,
                                       strides=(1,1),
                                       pad=True, name="conv5")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                       num_filters=256,
                                       strides=(1,1),
                                       pad=True, name='conv6')(h)
            h = C.layers.MaxPooling(filter_shape=(4,4),
                                    strides=(4,4), name="max3")(h)
            r = C.layers.Dense(num_output_classes, activation = None, name="classify")(h)
            return r

In [51]:
def create_model(features):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.relu):
            h = features

            h = C.layers.Convolution2D(filter_shape=(5,5),
                                        num_filters=32,
                                        strides=(1,1),
                                        pad=True, name="conv1")(h)
            h = C.layers.MaxPooling(filter_shape=(2,2),
                                        strides=(2,2), name="max1")(h)
            h = C.layers.Convolution2D(filter_shape=(5,5),
                                        num_filters=64,
                                        strides=(1,1),
                                        pad=True, name="conv2")(h)
            h = C.layers.MaxPooling(filter_shape=(3,3),
                                        strides=(3,3), name="max2")(h)
            h = C.layers.Dense(1024, activation = C.relu, name="fc1")(h)
            h = C.layers.Dropout(0.0)(h)
            r = C.layers.Dense(num_output_classes, activation = None, name="classify")(h)
            return r


In [37]:
# Scale the input to 0-1 range by dividing each pixel by 255.
#input_s = input/255

# used for LR model
#input_s = C.splice(x/255, C.square(x/255), C.sqrt(x/255))
#z = create_model(input_s)
#np.shape(input_s)

# used for MLP model
#z = create_model(x/255.0)

In [52]:
# Create the model
z = create_model(x)

# Print the output shapes / parameters of different components
print("Output Shape of conv1:", z.conv1.shape)
print("Output Shape of max1:", z.max1.shape)
print("Output Shape of conv2:", z.conv2.shape)
print("Output Shape of max2:", z.max2.shape)
print("Output Shape of fc1:", z.fc1.shape)
#print("Output Shape of conv3:", z.conv3.shape)
#print("Output Shape of conv4:", z.conv4.shape)
#print("Output Shape of conv5:", z.conv5.shape)
#print("Output Shape of conv6:", z.conv6.shape)
print("Output Shape of the dense output layer:", z.classify.shape)
print("Bias value of the last dense layer:", z.classify.b.value)

Output Shape of conv1: (32, 64, 64)
Output Shape of max1: (32, 32, 32)
Output Shape of conv2: (64, 32, 32)
Output Shape of max2: (64, 10, 10)
Output Shape of fc1: (1024,)
Output Shape of the dense output layer: (4,)
Bias value of the last dense layer: [0. 0. 0. 0.]


In [53]:
# Number of parameters in the network
C.logging.log_number_of_parameters(z)

Training 6610820 parameters in 8 parameter tensors.


## Training

In [54]:
def create_criterion_function(model, labels):
    loss = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error(model, labels)
    return loss, errs # (model, labels) -> (loss, error metric)

In [55]:
# Define a utility function to compute the moving average sum.
# A more efficient implementation is possible with np.cumsum() function
def moving_average(a, w=5):
    if len(a) < w:
        return a[:]    # Need to send a copy of the array
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx, val in enumerate(a)]


# Defines a utility that prints the training progress
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"

    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose: 
            print ("Minibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100))
        
    return mb, training_loss, eval_error

In [103]:
def train_test(train_reader, test_reader, model_func, num_sweeps_to_train_with=10):
    
    # Instantiate the model function; x is the input (feature) variable 
    # We will scale the input image pixels within 0-1 range by dividing all input value by 255.
    model = model_func(x/255)
    
    # Instantiate the loss and error function
    loss, label_error = create_criterion_function(model, y)
    
    # Instantiate the trainer object to drive the model training
    learning_rate = 0.1
    lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
    learner = C.sgd(z.parameters, lr_schedule)
    trainer = C.Trainer(z, (loss, label_error), [learner])
    
    # Initialize the parameters for the trainer
    minibatch_size = 64
    num_samples_per_sweep = 15000
    num_minibatches_to_train = (num_samples_per_sweep * num_sweeps_to_train_with) / minibatch_size
    
    # Map the data streams to the input and labels.
    input_map={
        y  : train_reader.streams.labels,
        x  : train_reader.streams.features
    } 
    
    # Uncomment below for more detailed logging
    training_progress_output_freq = 250
     
    # Start a timer
    start = time.time()

    for i in range(0, int(num_minibatches_to_train)):
        # Read a mini batch from the training data file
        data=train_reader.next_minibatch(minibatch_size, input_map=input_map) 
        trainer.train_minibatch(data)
        print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
     
    # Print training time
    print("Training took {:.1f} sec".format(time.time() - start))
    
    # Test the model
    test_input_map = {
        y  : test_reader.streams.labels,
        x  : test_reader.streams.features
    }

    # Test data for trained model
    test_minibatch_size = 250
    num_samples = 5000
    num_minibatches_to_test = num_samples // test_minibatch_size

    test_result = 0.0   

    for i in range(num_minibatches_to_test):
    
        # We are loading test data in batches specified by test_minibatch_size
        # Each data point in the minibatch is a MNIST digit image of 784 dimensions 
        # with one pixel per dimension that we will encode / decode with the 
        # trained model.
        data = test_reader.next_minibatch(test_minibatch_size, input_map=test_input_map)
        eval_error = trainer.test_minibatch(data)
        test_result = test_result + eval_error

    # Average of evaluation errors of all test minibatches
    print("Average test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test))

### Run training

In [104]:
def do_train_test():
    global z
    z = create_model(x)
    reader_train = create_reader(train_file, True, input_dim, num_output_classes)
    reader_test = create_reader(test_file, False, input_dim, num_output_classes)
    train_test(reader_train, reader_test, z)
    
do_train_test()

Minibatch: 0, Loss: 1.4072, Error: 75.00%
Minibatch: 250, Loss: 0.4135, Error: 12.50%
Minibatch: 500, Loss: 0.1362, Error: 6.25%
Minibatch: 750, Loss: 0.0185, Error: 0.00%
Minibatch: 1000, Loss: 0.0161, Error: 0.00%
Minibatch: 1250, Loss: 0.0180, Error: 0.00%
Minibatch: 1500, Loss: 0.0230, Error: 1.56%
Minibatch: 1750, Loss: 0.0028, Error: 0.00%
Minibatch: 2000, Loss: 0.0030, Error: 0.00%
Minibatch: 2250, Loss: 0.0195, Error: 1.56%
Training took 71.6 sec
Average test error: 0.46%


In [105]:
#only used for CNN model
print("Bias value of the last dense layer:", z.classify.b.value)

Bias value of the last dense layer: [ 0.09998371 -0.06732555 -0.00011704 -0.03254468]


## Evaluation

In [106]:
out = C.softmax(z)

In [107]:
# Read the data for evaluation
reader_eval=create_reader(test_file, False, input_dim, num_output_classes)

eval_minibatch_size = 5000
eval_input_map = {x: reader_eval.streams.features, y:reader_eval.streams.labels} 

data = reader_eval.next_minibatch(eval_minibatch_size, input_map=eval_input_map)

img_label = data[y].asarray()
img_data = data[x].asarray()

# reshape img_data to: M x 1 x 64 x 64 to be compatible with model
img_data = np.reshape(img_data, (eval_minibatch_size, 1, 64, 64))

predicted_label_prob = [out.eval(img_data[i]) for i in range(len(img_data))]

In [108]:
# Find the index with the maximum value for both predicted as well as the ground truth
pred = [np.argmax(predicted_label_prob[i]) for i in range(len(predicted_label_prob))]
gtlabel = [np.argmax(img_label[i]) for i in range(len(img_label))]
#print("Label    :", gtlabel[:5000])
#print("Predicted:", pred)

count = 0
for i in range(len(pred)):
    if pred[i] != gtlabel[i]:
        count += 1
print("Count: ", count)
print("Error: ", count / len(pred))

Count:  41
Error:  0.0082


## Submission

In [300]:
from PIL import Image
def loadImage(filename):
    img = Image.open(filename)
    pixels = img.load()
    img_data = ''
    for i in range(img.size[0]):    # for every col:
        for j in range(img.size[1]):    # For every row
            img_data = img_data + ' ' + str(pixels[i,j])
    return img_data

In [93]:
%%time
import os
from PIL import Image
test_dir = 'data/test'
test_files = os.listdir(test_dir)
label = '0 0 0 0'
feature = ''
output = []
print("Reading files...")
for file in test_files:
    img = Image.open(test_dir + '/' + file)
    img_data = np.array(img.getdata())
    img_data = np.reshape(img_data, (1, 1, 64, 64))
    predict_result = out.eval(img_data)
    orientation = [np.argmax(predict_result[i]) for i in range(len(predict_result))]
    id = file[:file.index('.')]
    output.append('{},{}\n'.format(id, orientation[0]))

import time
import datetime
filename = 'data/Eval-v1-3_' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S') + '.txt'
if not os.path.isfile(filename):
    print("Saving", filename )
    with open(filename, 'w') as f:
        for row in output:
            f.write(row)
    print("File save complete.")
else:
    print("File already exists", filename)

Reading files...


C:\Anaconda\lib\site-packages\cntk\core.py:350: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  elif np.issubdtype(sample.dtype, int):
C:\Anaconda\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "int32", but your input variable (uid "Input3") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


Saving data/Eval-v1-3_20180718_233202.txt
File save complete.
Wall time: 45.3 s
